In [7]:
!python -m pip install pymupdf pandas openpyxl tensorflow tensorflow_hub tensorflow-text spacy classy-classification 

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!python -m spacy download es_dep_news_trf

^C


# Resumen Ejecutivo
Durante el proceso de revisión de los informes de práctica del DISC (Departamento de Ingeniería de Sistemas y Computación), se requiere una inversión considerable de tiempo que, hasta la fecha, no ha sido automatizada. Esto conlleva largas jornadas de trabajo y carga adicional para los académicos, quienes podrían emplear ese tiempo en otras labores. Por lo tanto, como equipo de trabajo, hemos llegado a un consenso en la necesidad de llevar a cabo el análisis y desarrollo de un modelo que permita clasificar los informes en las categorías definidas en la rúbrica actual (insatisfactorio, regular, bueno y excelente).
Es importante destacar que, con la llegada de la pandemia, la entrega de informes ha sido en formato digital, lo que ha generado un conjunto de aproximadamente 100 informes disponibles. Esta digitalización ofrece ventajas significativas para el entrenamiento del modelo, ya que se dispone de datos de entrada y resultados concretos (informe, rúbrica y nota).


In [1]:
import fitz
import pandas as pd
#pd.set_option("mode.copy_on_write", True) #not on Python 3.9

# Lectura de datos


In [2]:
def get_classification(grade, number=False):
  classification = [0,0,0] # Regular, Bueno, Excelente (Todo 0 = Insatisfactorio)
  grade = round(grade, 1)
  if(grade < 4):
    return "insatisfactorio" if not number else 0
  elif (4 <= grade < 5.5):
    classification[0] = 1
    return "regular" if not number else 1
  elif (5.5 <= grade < 6.5):
    classification[1] = 1
    return "bueno" if not number else 2
  elif (6.5 <= grade <= 7):
    classification[2] = 1
    return "excelente" if not number else 3

In [58]:
dataset = pd.read_excel("calificaciones.xlsx", decimal=',')
grades_columns = dataset.columns.difference(["id", "periodo", "Unnamed: 9"]) #["estructura", "escritura", "contenido", "conclusiones", "conocimiento", "relevancia", "total"]
rubric_columns = grades_columns.difference(["total"]) #, "escritura", "estructura"
dataset = dataset.dropna(subset=grades_columns)

# Extracción y limpieza de documentos
En esta sección, se cargan los documentos en formato PDF, para la extracción y limpieza de estos, seguido de su integración al dataset.

In [4]:
documents = []

for id in dataset['id']:
    pdf_file = fitz.open(f"dataset/{id}.pdf")
    document_text = chr(12).join([page.get_text() for page in pdf_file])
    documents.append(document_text)

dataset.insert(loc=2, column="documents", value=documents)
dataset

,id,periodo,documents,estructura,escritura,contenido,conclusiones,conocimiento,relevancia,total,Unnamed: 9
0,20908397-1,2023-1,\n \nUNIVERSIDAD CATÓLICA DEL NORTE \nFACUL...,6.2,5.1,6.0,5.5,4.4,6.0,5.3,NaN
1,18971994-1,2023-1,\nAntofagasta \n \n Abril de 2023 \...,6.9,6.8,6.8,6.8,7.0,6.9,6.9,NaN
2,19445943-1,2023-1,\n1 \n \n \nUNIVERSIDAD CATÓLICA DEL NORTE \n...,6.7,6.9,6.5,6.4,6.8,7.0,6.7,NaN
5,19463712-1,2023-1,\n \n \n \n ...,4.4,4.9,4.9,5.8,4.0,6.0,4.9,NaN
6,20218430-1,2023-1,\nUNIVERSIDAD CATÓLICA DEL NORTE \nFACULTAD D...,6.1,5.8,5.5,5.0,4.5,5.8,5.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...
177,19928371-1,2021-1,\nUNIVERSIDAD CATÓLICA DEL NORTE \nFACULTAD D...,6.8,6.2,6.0,5.7,6.3,6.0,6.1,NaN
178,19952605-1,2021-1,\n \n \n \nUNIVERSIDAD CATÓLICA DEL NORTE \nF...,7.0,6.8,6.8,7.0,7.0,7.0,6.9,NaN
179,19957163-1,2021-1,\n \nUNIVERSIDAD CATÓLICA DEL NORTE \nFACULTA...,6.5,4.5,5.8,5.5,6.4,6.3,5.8,NaN
180,20180533-1,2021-1,\nUNIVERSIDAD CATÓLICA DEL NORTE \nFACULTAD D...,7.0,6.0,6.7,6.8,6.8,7.0,6.7,NaN


# Preprocesamiento de etiquetas
Con base en las notas obtenidas por cada entrada, se clasifican los documentos en las categorías definidas en la rúbrica.
Se consideran dos tipos de etiquetas: texto y número. El primero se utiliza para el entrenamiento de modelos de Deep Learning, mientras que el segundo se utiliza para el entrenamiento de modelos tradicionales.

In [5]:
text_labeled_dataset = dataset.copy()
text_labeled_dataset.loc[:, grades_columns] = text_labeled_dataset.loc[:, grades_columns].apply(lambda s: s.apply(get_classification))
dataset.loc[:, grades_columns] = dataset.loc[:, grades_columns].apply(lambda s: s.apply(lambda x: get_classification(grade=x, number=True)))

C:\Users\jose6\AppData\Local\Temp\ipykernel_22860\4211292860.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dataset.loc[:, grades_columns] = dataset.loc[:, grades_columns].apply(lambda s: s.apply(lambda x: get_classification(grade=x, number=True)))


# Clasificación de documentos con métodos tradicionales
Se usará SciKit-Learn para el análisis de los documentos mediante métodos tradicionales de NLP, como TF-IDF ~~y Word2Vec?~~. Se utilizará un modelo de clasificación de regresión logística y SVM para la clasificación de los documentos.
En primera instancia, solo se probará la clasificiación final.

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier, LogisticRegression, LinearRegression
from sklearn.naive_bayes import MultinomialNB, CategoricalNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, f1_score

## Preprocesamiento TF-IDF

In [7]:
Xn = dataset['documents']
yn = dataset[grades_columns]
X_train, X_test, y_train, y_test = train_test_split(Xn, yn, random_state=3, test_size=0.3)

In [8]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_df=0.9, min_df=0.2)
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

## Regresores de nota final

In [66]:
def get_final_grade(rubric_grades):
    result = rubric_grades['estructura']*0.5
    result += rubric_grades['escritura']*0.15
    result += rubric_grades['contenido']*0.25
    result += rubric_grades['conclusiones']*0.15
    result += rubric_grades['conocimiento']*0.30
    result += rubric_grades['relevancia']*0.10
    return round(result)

In [73]:
y_calc = get_final_grade(yn[rubric_columns])
print(accuracy_score(y_calc, yn['total']))
y_calc

0.10344827586206896


0      2.0
1      4.0
2      4.0
5      2.0
6      2.0
      ... 
177    3.0
178    4.0
179    3.0
180    4.0
181    4.0
Name: estructura, Length: 174, dtype: float64

In [59]:
final_grade_rf = RandomForestClassifier()
final_grade_rf.fit(y_train[rubric_columns], y_train['total'])
grade_pred_rf = final_grade_rf.predict(y_test[rubric_columns])
print(accuracy_score(y_test['total'], grade_pred_rf))
{'Característica': rubric_columns, 'Importancia': final_grade_rf.feature_importances_}

0.8113207547169812


{'Característica': Index(['conclusiones', 'conocimiento', 'contenido', 'escritura', 'estructura',
        'relevancia'],
       dtype='object'),
 'Importancia': array([0.16857424, 0.2603969 , 0.26508912, 0.19325956, 0.05499339,
        0.05768678])}

In [60]:
final_grade_log = LogisticRegression(max_iter=1000)
final_grade_log.fit(y_train[rubric_columns], y_train['total'])
grade_pred_log = final_grade_log.predict(y_test[rubric_columns])
print(accuracy_score(y_test['total'], grade_pred_log))

0.8679245283018868


{'Característica': Index(['conclusiones', 'conocimiento', 'contenido', 'escritura', 'estructura',
        'relevancia'],
       dtype='object'),
 'Importancia': array([[-1.34695751, -1.35070132, -0.92570698, -1.02865765, -0.25962928,
         -1.29684428],
        [-0.59156168, -1.05596707, -0.89826997, -1.19481917, -0.23909476,
         -0.17719915],
        [ 0.46610424,  0.36809972,  0.03083842,  0.40622851, -0.19707045,
          0.37408921],
        [ 1.47241495,  2.03856867,  1.79313853,  1.8172483 ,  0.69579448,
          1.09995422]])}

In [1]:
final_grade_nb = CategoricalNB()
final_grade_nb.fit(y_train[rubric_columns], y_train['total'])
grade_pred_nb = final_grade_log.predict(y_test[rubric_columns])
print(accuracy_score(y_test['total'], grade_pred_nb))
print(f1_score(y_test['total'], grade_pred_nb, average='weighted'))

NameError: name 'CategoricalNB' is not defined

## Clasificación de documentos con SVM

In [85]:
svc_n = SVC(C=10)
svc_n.fit(X_train_bow, y_train['total'])
y_pred = svc_n.predict(X_test_bow)
print(accuracy_score(y_test['total'], y_pred))
print(f1_score(y_test['total'], y_pred, average='weighted'))

0.5849056603773585
0.5516265369466873


In [80]:
svc_n_mo = MultiOutputClassifier(svc_n)
svc_n_mo.fit(X_train_bow, y_train[rubric_columns])
y_pred_cont = svc_n_mo.predict(X_test_bow)
print(svc_n_mo.score(X_test_bow, y_test[rubric_columns]))
print(f1_score(y_test[rubric_columns], y_pred_cont, average='weighted'))
y_pred_cont

0.07547169811320754


ValueError: multiclass-multioutput is not supported

## Clasificación de documentos con Regresión Ridge

In [84]:
ridge = RidgeClassifier()
ridge.fit(X_train_bow, y_train['total'])
y_pred_r = ridge.predict(X_test_bow)
print(accuracy_score(y_test['total'], y_pred_r))
print(f1_score(y_test['total'], y_pred_r, average='weighted'))

0.5660377358490566
0.5275254892769249


In [40]:
ridge_mo = MultiOutputClassifier(ridge)
ridge_mo.fit(X_train_bow, y_train[rubric_columns])
y_pred_cont = ridge_mo.predict(X_test_bow)
print(ridge_mo.score(X_test_bow, y_test[rubric_columns]))
y_pred_cont

0.1320754716981132


array([[2, 1, 3, 2],
       [3, 2, 2, 3],
       [2, 2, 2, 2],
       [3, 1, 2, 3],
       [3, 3, 2, 3],
       [3, 2, 2, 2],
       [3, 3, 2, 3],
       [2, 2, 2, 3],
       [3, 3, 3, 3],
       [3, 2, 2, 3],
       [2, 1, 1, 3],
       [2, 1, 2, 2],
       [2, 3, 2, 3],
       [2, 2, 2, 3],
       [2, 2, 2, 3],
       [3, 3, 1, 3],
       [3, 1, 2, 2],
       [2, 3, 2, 3],
       [3, 2, 2, 3],
       [3, 1, 1, 3],
       [1, 2, 2, 2],
       [1, 1, 1, 3],
       [3, 2, 2, 3],
       [3, 2, 2, 3],
       [2, 1, 1, 2],
       [3, 3, 3, 3],
       [3, 3, 2, 3],
       [3, 2, 2, 3],
       [3, 3, 3, 3],
       [2, 3, 2, 3],
       [1, 2, 2, 2],
       [3, 3, 2, 3],
       [3, 3, 2, 2],
       [3, 2, 2, 3],
       [2, 2, 2, 2],
       [3, 3, 2, 3],
       [2, 3, 2, 3],
       [3, 1, 2, 2],
       [2, 2, 2, 3],
       [3, 1, 2, 2],
       [3, 3, 2, 3],
       [3, 2, 2, 3],
       [1, 2, 2, 2],
       [3, 3, 2, 3],
       [2, 3, 2, 3],
       [2, 2, 2, 2],
       [3, 2, 2, 3],
       [2, 3,

## Clasificación de documentos con Regresión Logística

In [83]:
log = LogisticRegression()
log.fit(X_train_bow, y_train['total'])
y_pred_l = log.predict(X_test_bow)
print(accuracy_score(y_test['total'], y_pred_l))
print(f1_score(y_test['total'], y_pred_l, average='weighted'))

0.6037735849056604
0.5322023148882195


## Clasificación de documentos con Random Forest

In [13]:
rf = RandomForestClassifier()
rf.fit(X_train_bow, y_train['total'])
y_pred_rf = log.predict(X_test_bow)
print(accuracy_score(y_test['total'], y_pred_l))

0.6037735849056604


In [75]:
rf_mo = MultiOutputClassifier(rf)
rf_mo.fit(X_train_bow, y_train[rubric_columns])
y_pred_cont = rf_mo.predict(X_test_bow)
print(rf_mo.score(X_test_bow, y_test[rubric_columns]))
y_pred_cont

0.018867924528301886


array([[2, 1, 2, 2, 2, 3],
       [3, 1, 2, 1, 3, 3],
       [3, 2, 2, 2, 3, 3],
       [3, 1, 2, 2, 2, 3],
       [3, 3, 2, 1, 3, 3],
       [3, 3, 3, 2, 3, 3],
       [3, 3, 2, 2, 3, 3],
       [2, 2, 2, 2, 2, 3],
       [2, 3, 2, 1, 3, 3],
       [3, 1, 2, 1, 3, 3],
       [2, 1, 2, 2, 3, 3],
       [2, 3, 2, 2, 3, 3],
       [3, 3, 2, 1, 3, 3],
       [3, 2, 2, 2, 3, 3],
       [2, 2, 2, 2, 3, 3],
       [2, 1, 2, 1, 3, 3],
       [3, 3, 2, 2, 3, 3],
       [3, 3, 2, 1, 3, 3],
       [3, 2, 2, 1, 3, 3],
       [2, 1, 2, 1, 3, 3],
       [1, 1, 2, 1, 2, 2],
       [3, 3, 2, 2, 3, 3],
       [3, 3, 2, 2, 3, 3],
       [2, 3, 2, 2, 3, 2],
       [2, 1, 2, 2, 3, 2],
       [2, 2, 2, 1, 3, 3],
       [3, 1, 2, 1, 3, 3],
       [2, 3, 2, 2, 3, 3],
       [2, 2, 3, 1, 3, 3],
       [3, 3, 2, 2, 3, 3],
       [3, 2, 2, 1, 3, 2],
       [3, 2, 2, 1, 3, 3],
       [3, 3, 2, 2, 3, 2],
       [3, 2, 2, 1, 3, 3],
       [2, 1, 1, 1, 3, 2],
       [3, 3, 2, 2, 3, 3],
       [3, 3, 2, 1, 3, 3],
 

## Clasificación de documentos con Naïve Bayes

In [16]:
nb = MultinomialNB()
nb.fit(X_train_bow, y_train['total'])
y_pred_nb = nb.predict(X_test_bow)
print(accuracy_score(y_test['total'], y_pred_nb))

0.5660377358490566


# Clasificación de documentos con Deep Learning

## Clasificación de documentos con Spacy

In [ ]:
Xn_text_labeled = text_labeled_dataset['documents']
yn_text_labeled = text_labeled_dataset['total']
X_text_train, X_text_test, y_text_train, y_text_test = train_test_split(Xn_text_labeled, yn_text_labeled, random_state=3, test_size=0.3)

In [ ]:
training_data_total = {
  "insatisfactorio": [],
  "regular": [],
  "bueno": [],
  "excelente": []
}
for index, document in X_text_train.items():
  training_data_total[y_text_train[index]].append(document)
training_data_total

https://spacy.io/universe/project/classyclassification

In [ ]:
from classy_classification import ClassyClassifier
classifier = ClassyClassifier(data=training_data_total)
classifier.set_embedding_model(model="paraphrase-multilingual-mpnet-base-v2")
y_text_pred = classifier.pipe(X_text_test.tolist())

In [ ]:
y_text_pred

In [ ]:
y_text_test

## Clasificación de documentos con TensorFlow

In [ ]:
import time
from keras.layers import Embedding, LSTM, Bidirectional, Dense
from keras.models import Sequential
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub
from keras import utils
PARAGRAPH_QTY = 512
BATCH_SIZE= 171
labels = utils.to_categorical(dataset['total'], num_classes=4)


### Clasificación de documentos con Universal Sentence Encoder - Multilingual Large
La estructura de entrada corresponde a la siguiente:
Cada documento se encuentra almacenado como una lista de párrafos.
Así, la entrada del modelo LSTM sería un conjunto de lotes, correspondiendo a lo

In [ ]:
use_embedder = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [ ]:
time_start = time.perf_counter()
Xn_text_embed = use_embedder(dataset['documents'])
time_end = time.perf_counter()
print(time_end-time_start)

In [ ]:
model_use = Sequential()
model_use.add(Bidirectional(LSTM(512, return_sequences=True), input_shape=(PARAGRAPH_QTY, 512)))
model_use.add(Dense(256, activation='relu'))
model_use.add(Dense(4, activation='softmax'))

In [ ]:
model_use.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy", "f1"])
model_use.fit(Xn_text_embed, yn, batch_size=64, validation_split=0.3, epochs=1)

### Clasificación de documentos con Longformer - Spanish

### Clasificación de documentos con Tulio BERT